In [ ]:
!pip install --quiet langchain
!pip install --quiet litellm
!pip install --quiet  langchain-google-genai
!pip install --quiet crewai crewai-tools
!pip install --quiet beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.0 MB/s eta 0:0

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from datetime import datetime
import requests
import json
from litellm import completion
import os

from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool

In [ ]:
google_api_key = userdata.get('GOOGLE_API_KEY')
# llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',
#                                api_key=google_api_key,
#                                temperature=0.5)
# llm.invoke('2+2').content
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.7,
)


In [ ]:
class AINewsLetterAgent():
  def editor_agent(self):
    return Agent(
        role='Editor',
        goal='Oversee the creation of the AI Newsletter',
        backstory='With a keen eye for detail and passion for storytelling. you \
        ensure that the newsletter not only informs but also engages and inspires \
        the readers',
        allow_delegation=True,
        verbose=True,
        max_iter=15, # ensure that it doesn't goes in endless loop to write newsletter
        llm=llm
    )

  def news_fetcher_agent(self):
    return Agent(
        role='News Fetcher',
        goal='Fetch the top AI news stories for the day',
        backstory='As a digital sleuth, you scoure internet for the latest and most \
        impactful developments in the world of AI, ensuring that our readers \
        are always in the know.',
        tools=[SearchTool.search_internet],
        verbose=True,
        allow_delegation=True,
        llm=llm
    )

  def news_analyzer_agent(self):
    return Agent(
        role='News Analyzer',
        goal='Analyze each new story and generate a detailed markdown summary',
        backstory='With a critical eye and a knack for distilling complex information \
        you analyses of AI news stories, making them accessible and engagin for\
        audience.',
        tools=[SearchTool.search_internet],
        verbose=True,
        allow_delegation=True,
        llm=llm
    )

  def news_letter_compiler_agent(self):
    return Agent(
        role="Newsletter Compiler",
        goal="Compile the analyzed news stories into a final newsletter format",
        backstory="As the final architect of the newsletter, you meticulously\
        arrange and ensure a coherent and visually appealing presentation that \
        captivates our reader. Ensure newsletter format guidelines and maintain \
        consistency throughout",
        verbose=True,
        llm=llm
    )

In [ ]:
class AINewsLetterTasks():
  def fetch_news_task(self, agent):
    return Task(
        description=f"Fetch top AI news stories from the past 24 hours. The current \
        time is {datetime.now()}",
        agent=agent,
        async_execution=True,
        expected_output="""A list of top AI news story titles, URLs, and a brief
        summary for each story from them
        Expected Output:
        [
          {'title': 'AI takes spotlight in Super Bowl Commercials',
          'url': 'https://example.com/story1',
          'summary': 'AI made a splash in this year\'s Super Bowl commercials..'
          },
          {(...)}
        ]

        """
    )

  def analyze_news_task(self, agent, context):
    return Task(
        description="Analyze each news story and generate ensure there are at least \
        5 well formated articles",
        agent=agent,
        context=context,
        expected_output=""" A markdown-formatted analysis for each news story, including\
        a rundown, detailed bullet points and a 'Why it matters' section. There should \
        be atleast 5 articles, each following the proper format.
        Example output:
        ## AI takes spotlight in Super Bowl commercials \n\n
        ** The Rundown:
        ** AI made a splash in this year\'s Superbowl commercials... \n\n
        ** The details: ** \n\n
        - Microsoft\'s Copilot spot showcased its AI assistant ... \n\n
        **Why it matters:** While AI-related ads have been rampant over the last year \n,
        its Super Bowl presence ...
        """,
    )

  def compile_newsletter_task(self, agent, context, call_back_function):
    return Task(
        description="Compile the analyzed news letter",
        agent=agent,
        context=context,
        expected_output="""A complete newsletter in markdown format, with a consistent \
        style and layout. Example output:
        # Top stories in AI today: \\n \\n
        - AI takes spotlight in Super Bowl Commercials \\n
        - Altman seeks triilions for global AI chip initiative \\n \\n

        ## AI takes spotlight in Super Bowl commercials \\n \\n
        ** The Rundown: ** \\n \\n
        ** Why it matters:: ** ... \\n \\n
        ## Altman seeks trillions for global AI chip initiative \\n \\n
        ** The Rundown: ** \\n \\n
        ** Why it matters:: ** ... \\n \\n
        """,
        call_back_function=call_back_function,
        )

In [ ]:
serper_api_key = userdata.get('SERPER_API_KEY')

class SearchTool():
  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet about a given topic and return relevant results"""
    print("Searching the internet")
    top_result_to_return = 5
    url = "https://google.serper.dev/search"
    payload = json.dumps(
        {
            "q": query,
            "num": top_result_to_return,
            "tbm": "nws"
        }
    )
    headers = {
        'X-API-KEY': serper_api_key,
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    if 'organic' not in response.json():
      return "Sorry I couldn't find anything about that, there could be an error \
      with your serper api key."
    else:
      results = response.json()['organic']
      string = []
      print(f"Results: {results[:top_result_to_return]}")
      for result in results[:top_result_to_return]:
        try:
          data = result.get('date', 'Date not available')
          string.append('\n'.join([
                        f'Title: {result["title"]}',
                        f'Link: {result["link"]}',
                        f'Snippet: {result["snippet"]}',
                        f'Date: {data}',
                        "\n----------------------------",
                    ]))
        except:
          print("error")
      return '\n'.join(string)

In [ ]:
def save_markdown(task_output):
  today_date = datetime.now().strftime("%Y-%m-%d")
  filename = f"{today_date}.md"
  with open(filename, "w") as f:
    f.write(task_output.result)
  print(f"Saved markdown file: {filename}")


In [ ]:
agents = AINewsLetterAgent()
tasks = AINewsLetterTasks()

# setting up agents
editor = agents.editor_agent()
news_fetcher = agents.news_fetcher_agent()
news_analyzer = agents.news_analyzer_agent()
newsletter_compiler = agents.news_letter_compiler_agent()

fetch_news_task = tasks.fetch_news_task(news_fetcher)
analyze_news_task = tasks.analyze_news_task(agent=news_analyzer, context=[fetch_news_task])
compiled_newsletter_task = tasks.compile_newsletter_task(agent=newsletter_compiler, context=[analyze_news_task], call_back_function=save_markdown)
crew = Crew(
    agents=[editor, news_fetcher, news_analyzer, newsletter_compiler],
    tasks=[fetch_news_task, analyze_news_task, compiled_newsletter_task],
    process=Process.hierarchical,
    manager_llm=llm,
    verbose=True,
)

results = crew.kickoff()
print(f"Crew work results: {results}")

In [ ]:
results.raw

"# Top Stories in AI Today:\n\n- AI-Powered Drug Discovery Breakthrough\n- Concerns Rise Over AI-Generated Deepfakes\n- AI Bias in Hiring Algorithms Under Scrutiny\n- EU Parliament Votes on New AI Regulations\n- Microsoft Integrates AI Across its Product Suite\n\n\n## AI-Powered Drug Discovery Breakthrough\n\n**The Rundown:** A research team announced a significant advancement in AI-powered drug discovery, identifying a potential treatment for a rare genetic disease.\n\n**The Details:**\n\n* Researchers utilized a novel deep learning model to analyze vast datasets of genomic information and molecular structures.\n* The AI identified a previously unknown molecule with high efficacy against the target disease.\n* Pre-clinical trials are underway to validate the AI's findings.\n\n**Why it matters:** This showcases the growing potential of AI to accelerate and revolutionize the drug discovery process, potentially leading to faster development of life-saving treatments.\n\n\n## Concerns Ris

In [ ]:

from IPython.display import Markdown

def display_markdown(markdown_text):
  """
  Displays markdown text as markdown in a Jupyter environment.
  """
  display(Markdown(markdown_text))

display_markdown(results.raw)

# Top Stories in AI Today:

- AI-Powered Drug Discovery Breakthrough
- Concerns Rise Over AI-Generated Deepfakes
- AI Bias in Hiring Algorithms Under Scrutiny
- EU Parliament Votes on New AI Regulations
- Microsoft Integrates AI Across its Product Suite


## AI-Powered Drug Discovery Breakthrough

**The Rundown:** A research team announced a significant advancement in AI-powered drug discovery, identifying a potential treatment for a rare genetic disease.

**The Details:**

* Researchers utilized a novel deep learning model to analyze vast datasets of genomic information and molecular structures.
* The AI identified a previously unknown molecule with high efficacy against the target disease.
* Pre-clinical trials are underway to validate the AI's findings.

**Why it matters:** This showcases the growing potential of AI to accelerate and revolutionize the drug discovery process, potentially leading to faster development of life-saving treatments.


## Concerns Rise Over AI-Generated Deepfakes

**The Rundown:** A new report highlights the increasing sophistication and ease of creating realistic deepfake videos, raising concerns about their potential misuse in misinformation campaigns.

**The Details:**

* Advances in generative AI models allow for the creation of increasingly convincing deepfakes, even with limited training data.
* Experts warn of the potential for deepfakes to be used to damage reputations, spread propaganda, and undermine public trust.
* Discussions are underway regarding the need for stricter regulations and improved detection technologies.

**Why it matters:** The proliferation of deepfakes poses a significant threat to societal trust and stability. The development of effective countermeasures is crucial.


## AI Bias in Hiring Algorithms Under Scrutiny

**The Rundown:** A study revealed significant bias in AI-powered hiring algorithms, disproportionately favoring certain demographic groups over others.

**The Details:**

* The study analyzed data from several major companies using AI for candidate screening.
* Results indicated that the algorithms exhibited biases related to gender, race, and socioeconomic background.
* Experts are calling for greater transparency and accountability in the development and deployment of AI hiring tools.

**Why it matters:** Biased AI systems can perpetuate and amplify existing societal inequalities. Addressing algorithmic bias is essential to ensure fairness and equity in the hiring process.


## EU Parliament Votes on New AI Regulations

**The Rundown:** The European Parliament voted to adopt a landmark bill regulating the development and deployment of AI systems within the European Union.

**The Details:**

* The AI Act aims to classify AI systems based on their risk level and impose different regulatory requirements accordingly.
* High-risk AI systems, such as those used in critical infrastructure or law enforcement, will face stricter scrutiny.
* The legislation includes provisions for transparency, accountability, and human oversight.

**Why it matters:** The EU AI Act represents a significant step towards establishing a global framework for responsible AI development. Its impact could influence regulatory efforts in other parts of the world.


## Microsoft Integrates AI Across its Product Suite

**The Rundown:** Microsoft announced further integration of AI capabilities across its range of products, including Office 365 and Windows.

**The Details:**

* New AI-powered features are being rolled out to improve productivity, accessibility, and user experience.
* This includes enhancements to text editing, image generation, and data analysis tools.
* The company is emphasizing responsible AI development and user privacy.

**Why it matters:** The widespread integration of AI into everyday software applications will significantly impact how people work, communicate, and interact with technology.